In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.1 MB/s 
     |████████████████████████████████| 182 kB 51.1 MB/s 
     |████████████████████████████████| 7.6 MB 67.4 MB/s 


In [2]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
import re
import os
import urllib.request
from tqdm import tqdm
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [4]:
# GPU 확인하기
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [5]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#데이터 불러오기, 전처리

In [6]:
# data = pd.read_csv('/content/drive/MyDrive/데비_프로젝트/데이터/concat_all_notTokenized.csv')
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv", filename="finance_data.csv")
data = pd.read_csv('finance_data.csv')
data['labels'] = data['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
del data['sentence']

# 중복 제거
data.drop_duplicates(subset=['kor_sentence'], inplace=True)

In [7]:
data

,labels,kor_sentence
0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
...,...,...
4841,2,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842,0,린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843,2,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,2,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [8]:
# CLS, SEP 붙이기 (문장의 시작, 끝)
sentences = ["[CLS] " + str(s) + " [SEP]" for s in data.kor_sentence]
sentences[:3]

['[CLS] Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로 옮길 계획이 없다고 한다. [SEP]',
 '[CLS] 테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평방미터 이상의 면적을 단계적으로 개발할 계획이라고 성명은 밝혔다. [SEP]',
 '[CLS] 국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 해고와는 달리 회사는 사무직 직원 수를 줄였다고 일간 포스티메스가 보도했다. [SEP]']

In [9]:
labels = data['labels'].values
labels

array([0, 0, 2, ..., 2, 2, 2])

In [10]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('안녕하세요!')
print(result)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

['안', '##녕', '##하', '##세', '##요', '!']


In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

In [12]:
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로 옮길 계획이 없다고 한다. [SEP]
['[CLS]', 'Gran', '##에', '따르면', ',', '그', '회', '##사는', '회', '##사가', '성', '##장', '##하고', '있는', '곳', '##이지', '##만', ',', '모든', '생', '##산', '##을', '러시아', '##로', '옮', '##길', '계', '##획', '##이', '없다', '##고', '한다', '.', '[SEP]']


In [13]:
MAX_LEN = 60 #최대 시퀀스 길이 설정
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [14]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [15]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [16]:
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [17]:
# X_data = data['kor_sentence']
# y_data = data['label']

In [18]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2000, 
                                                                                    test_size=0.1)

In [19]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2000, 
                                                       test_size=0.1)

In [20]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [21]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [22]:
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

# 라벨 데이터
labels = data['labels'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 5.모델 생성

In [23]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model.cuda()

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [25]:
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 1

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


#6.모델 학습

In [26]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [27]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 1 ========
Training...

  Average training loss: 0.73
  Training epcoh took: 0:00:49

Running Validation...
  Accuracy: 0.70
  Validation took: 0:00:02

Training complete!


# 7.테스트셋 평가

In [28]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    151.    Elapsed: 0:00:24.

Accuracy: 0.76
Test took: 0:00:36


# 8. 뉴스 기사 감성분석

필요한 함수 만들기

In [29]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 60

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [30]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

# 분석할 뉴스 가져오기

In [138]:
news_df = pd.read_csv('/content/drive/MyDrive/데비_프로젝트/데이터/30days_3pages_samsung_news.csv')
news_df = news_df[['date','title']]
news_df.head()

,date,title
0,2022.08.01.,산짜이(山寨)문화
1,2022.08.01.,“네이버쇼핑 활용한 사기 및 개인정보 유출 심각”
2,2022.08.01.,"[마감] 삼성전자, 약보합세 마감...초등학교 입학 연령 하향 추진에 교육주 강..."
3,2022.08.01.,정부 계획에 맞춰 울산도 반도체 투자 시동
4,2022.08.01.,지정학적 리스크에 신중해진 K칩


In [139]:
def predict(sentences):
  logits = test_sentences([sentences])
  if np.argmax(logits) == 0 :
    sent = "0"
  elif np.argmax(logits) == 1 :
    sent = "1"
  elif np.argmax(logits) == 2 :
    sent = "2"
    
  return sent

news_df['predict'] = 0

for i in range(len(news_df)):
  sent = predict(news_df.iloc[i,1])
  news_df['predict'][i] = sent

news_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,date,title,predict
0,2022.08.01.,산짜이(山寨)문화,0
1,2022.08.01.,“네이버쇼핑 활용한 사기 및 개인정보 유출 심각”,0
2,2022.08.01.,"[마감] 삼성전자, 약보합세 마감...초등학교 입학 연령 하향 추진에 교육주 강...",1
3,2022.08.01.,정부 계획에 맞춰 울산도 반도체 투자 시동,0
4,2022.08.01.,지정학적 리스크에 신중해진 K칩,1
...,...,...,...
895,2022.08.30.,"[박신자컵]韓 여자농구가 기대하는 재능, 삼성생명 이해란의 구슬땀",0
896,2022.08.30.,[금융in] ‘금리인하요구 수용률 성적’ 공개됐다… 각 업권 수용률·감면액 ...,1
897,2022.08.30.,주택시장 '하이엔드 브랜드' 요구 날로 커져,1
898,2022.08.30.,"농협울산유통센터, 추석맞이 특별할인 행사 개최",0


전처리

In [140]:
# news_df = news_df[news_df['date'] != '3분 전']
# news_df = news_df[news_df['date'] != '5분 전']
# news_df = news_df[news_df['date'] != '2분 전']
# news_df = news_df[news_df['date'] != '4분 전']

# Results

Ratio of Sentiments (ALL DATA)

In [141]:
neutral = news_df[news_df['predict'] == 0]
positive = news_df[news_df['predict'] == 1]
negative = news_df[news_df['predict'] == 2]

# Neutral Ratio
Nt_R = len(neutral)/len(news_df)*100
# positive Ratio
Po_R = len(positive)/len(news_df)*100
# negative Ratio
Ne_R = len(negative)/len(news_df)*100

print('ratio of neutral:', Nt_R)
print('ratio of positive:', Po_R)
print('ratio of negative:', Ne_R)

ratio of neutral: 56.888888888888886
ratio of positive: 41.333333333333336
ratio of negative: 1.7777777777777777


### Ratio of Sentiment (PER DATE)

In [142]:
print('[Crawling Period]')
print(news_df['date'].iloc[0], '~',news_df['date'].iloc[-1])

[Crawling Period]
2022.08.01. ~ 2022.08.30.


Date to List

In [143]:
l = news_df['date'].value_counts()
l.to_frame()
date_list = list(l.index)
date_list

['2022.08.01.',
 '2022.08.02.',
 '2022.08.29.',
 '2022.08.28.',
 '2022.08.27.',
 '2022.08.26.',
 '2022.08.25.',
 '2022.08.24.',
 '2022.08.23.',
 '2022.08.22.',
 '2022.08.21.',
 '2022.08.20.',
 '2022.08.19.',
 '2022.08.18.',
 '2022.08.17.',
 '2022.08.16.',
 '2022.08.15.',
 '2022.08.14.',
 '2022.08.13.',
 '2022.08.12.',
 '2022.08.11.',
 '2022.08.10.',
 '2022.08.09.',
 '2022.08.08.',
 '2022.08.07.',
 '2022.08.06.',
 '2022.08.05.',
 '2022.08.04.',
 '2022.08.03.',
 '2022.08.30.']

Ratio of Sentiment (Date)

In [163]:
RatioSent_df = pd.DataFrame(columns=['date','neutral(%)','positive(%)','negative(%)'])
num = 0
for i in date_list:
  date_df = news_df[news_df['date'] == i]

  neutral = date_df[date_df['predict'] == 0]
  positive = date_df[date_df['predict'] == 1]
  negative = date_df[date_df['predict'] == 2]

  total = len(positive) + len(neutral) + len(negative)

  # Neutral Ratio
  Nt_R = len(neutral)/total*100
  # positive Ratio
  Po_R = len(positive)/total*100
  # negative Ratio
  Ne_R = len(negative)/total*100

  RatioSent_df.loc[num] = [i,Nt_R,Po_R,Ne_R]
  num = num + 1

RatioSent_df

,date,neutral(%),positive(%),negative(%)
0,2022.08.01.,63.333333,36.666667,0.000000
1,2022.08.02.,96.666667,3.333333,0.000000
2,2022.08.29.,60.000000,30.000000,10.000000
3,2022.08.28.,53.333333,40.000000,6.666667
4,2022.08.27.,50.000000,50.000000,0.000000
5,2022.08.26.,43.333333,56.666667,0.000000
6,2022.08.25.,46.666667,53.333333,0.000000
7,2022.08.24.,40.000000,60.000000,0.000000
8,2022.08.23.,50.000000,50.000000,0.000000
9,2022.08.22.,60.000000,40.000000,0.000000


# Weighted Average Sent From date

긍정 or 부정만 판단할 것이므로 중립의 비율은 무시.

부정보다 긍정의 비율이 다 높게 나오는 것으로 보임.

긍정 or 부정의 판단 방법:

if ) (부정비율) > 0 ⇒ 부정

if ) (부정비율) < 0 ⇒ 긍정

In [164]:
RatioSent_df['ratio'] = None
for i in range(len(RatioSent_df)):
  if RatioSent_df['positive(%)'].iloc[i] > 10 * float(RatioSent_df['negative(%)'].iloc[i]):
    RatioSent_df['ratio'].iloc[i] = RatioSent_df['positive(%)'].iloc[i]
  else:
    RatioSent_df['ratio'].iloc[i] = - float(RatioSent_df['negative(%)'].iloc[i])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [165]:
RatioSent_df['predict'] = None

for i in range(len(RatioSent_df)):
  if RatioSent_df['ratio'].iloc[i] > 45:
    RatioSent_df['predict'].iloc[i] = '긍정'
  else:
    RatioSent_df['predict'].iloc[i] = '부정'

RatioSent_df.head()

,date,neutral(%),positive(%),negative(%),ratio,predict
0,2022.08.01.,63.333333,36.666667,0.000000,36.666667,부정
1,2022.08.02.,96.666667,3.333333,0.000000,3.333333,부정
2,2022.08.29.,60.000000,30.000000,10.000000,-10.0,부정
3,2022.08.28.,53.333333,40.000000,6.666667,-6.666667,부정
4,2022.08.27.,50.000000,50.000000,0.000000,50.0,긍정


## 감성데이터전처리

In [166]:
RatioSent_df['date'] = [s.replace('.','/') for s in RatioSent_df['date']]
RatioSent_df['date'] = [s.rstrip('/') for s in RatioSent_df['date']]
RatioSent_df = RatioSent_df[['predict','date','ratio']]

## 주가 데이터 불러오기

In [167]:
stock_df = pd.read_csv('/content/drive/MyDrive/데비_프로젝트/데이터/삼성_50일주가데이터.csv',encoding='cp949')

In [168]:
# 필요한 데이터만
stock_df = stock_df[['일자','종가','대비','등락률','거래량']]
stock_df.rename(columns = {'일자' : 'date'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Date 기준으로 합치기 (뉴스 + 주가)

## 시도 1.

In [169]:
merge_df = pd.merge(RatioSent_df, stock_df, on='date')
merge_df

,predict,date,ratio,종가,대비,등락률,거래량
0,부정,2022/08/01,36.666667,61300,-100,-0.16,13154816
1,부정,2022/08/02,3.333333,61700,400,0.65,13614895
2,부정,2022/08/29,-10.0,58600,-1400,-2.33,9841329
3,긍정,2022/08/26,56.666667,60000,300,0.50,9499995
4,긍정,2022/08/25,53.333333,59700,700,1.19,5767902
5,긍정,2022/08/24,60.0,59000,-100,-0.17,8888486
6,긍정,2022/08/23,50.0,59100,-900,-1.50,9041629
7,부정,2022/08/22,40.0,60000,-900,-1.48,8259865
8,부정,2022/08/19,33.333333,60900,-600,-0.98,6923679
9,부정,2022/08/18,40.0,61500,1100,1.82,16372754


In [170]:
merge_df[merge_df['predict']=='부정']

,predict,date,ratio,종가,대비,등락률,거래량
0,부정,2022/08/01,36.666667,61300,-100,-0.16,13154816
1,부정,2022/08/02,3.333333,61700,400,0.65,13614895
2,부정,2022/08/29,-10.0,58600,-1400,-2.33,9841329
7,부정,2022/08/22,40.0,60000,-900,-1.48,8259865
8,부정,2022/08/19,33.333333,60900,-600,-0.98,6923679
9,부정,2022/08/18,40.0,61500,1100,1.82,16372754
13,부정,2022/08/11,43.333333,59900,800,1.35,15141941
15,부정,2022/08/09,-13.333333,60000,-800,-1.32,18251170
16,부정,2022/08/08,-13.333333,60800,-700,-1.14,11313150
18,부정,2022/08/04,43.333333,61500,200,0.33,9125439


In [171]:
merge_df[merge_df['predict']=='긍정']

,predict,date,ratio,종가,대비,등락률,거래량
3,긍정,2022/08/26,56.666667,60000,300,0.50,9499995
4,긍정,2022/08/25,53.333333,59700,700,1.19,5767902
5,긍정,2022/08/24,60.0,59000,-100,-0.17,8888486
6,긍정,2022/08/23,50.0,59100,-900,-1.50,9041629
10,긍정,2022/08/17,50.0,60400,-600,-0.98,9061518
11,긍정,2022/08/16,46.666667,61000,800,1.33,15036727
12,긍정,2022/08/12,56.666667,60200,300,0.50,10786658
14,긍정,2022/08/10,50.0,59100,-900,-1.50,18084349
17,긍정,2022/08/05,53.333333,61500,0,0.00,9567620
20,긍정,2022/08/30,46.666667,58800,200,0.34,9690044


# 모델평가

In [175]:
merge_df['등락'] = None
for i in range(len(merge_df)):
  if merge_df['등락률'].iloc[i] >= 0:
    merge_df['등락'].iloc[i] = 1 #상승
  else:
    merge_df['등락'].iloc[i] = 0 #하락

merge_df['pre'] = None
for i in range(len(merge_df)):
  if merge_df['predict'].iloc[i] == '긍정':
    merge_df['pre'].iloc[i] = 1 #상승
  else:
    merge_df['pre'].iloc[i] = 0 #하락

merge_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,predict,date,ratio,종가,대비,등락률,거래량,등락,pre
0,부정,2022/08/01,36.666667,61300,-100,-0.16,13154816,0,0
1,부정,2022/08/02,3.333333,61700,400,0.65,13614895,1,0
2,부정,2022/08/29,-10.0,58600,-1400,-2.33,9841329,0,0
3,긍정,2022/08/26,56.666667,60000,300,0.50,9499995,1,1
4,긍정,2022/08/25,53.333333,59700,700,1.19,5767902,1,1


## 모델평가

In [182]:
merge_df['yes/no'] = None
for i in range(len(merge_df)):
  if merge_df['등락'].iloc[i] == merge_df['pre'].iloc[i]:
    merge_df['yes/no'].iloc[i] = 'yes' # 정답
  else:
    merge_df['yes/no'].iloc[i] = 'no' #오답
merge_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,predict,date,ratio,종가,대비,등락률,거래량,등락,pre,yes/no
0,부정,2022/08/01,36.666667,61300,-100,-0.16,13154816,0,0,yes
1,부정,2022/08/02,3.333333,61700,400,0.65,13614895,1,0,no
2,부정,2022/08/29,-10.0,58600,-1400,-2.33,9841329,0,0,yes
3,긍정,2022/08/26,56.666667,60000,300,0.50,9499995,1,1,yes
4,긍정,2022/08/25,53.333333,59700,700,1.19,5767902,1,1,yes
5,긍정,2022/08/24,60.0,59000,-100,-0.17,8888486,0,1,no
6,긍정,2022/08/23,50.0,59100,-900,-1.50,9041629,0,1,no
7,부정,2022/08/22,40.0,60000,-900,-1.48,8259865,0,0,yes
8,부정,2022/08/19,33.333333,60900,-600,-0.98,6923679,0,0,yes
9,부정,2022/08/18,40.0,61500,1100,1.82,16372754,1,0,no


CONFUSION MATRICS

In [186]:
TP = merge_df[(merge_df['등락']==1)&(merge_df['pre']==1)]
FP = merge_df[(merge_df['등락']==0)&(merge_df['pre']==1)]
FN = merge_df[(merge_df['등락']==1)&(merge_df['pre']==0)]
TN = merge_df[(merge_df['등락']==0)&(merge_df['pre']==0)]


In [181]:
cm = {'예측값(Y)':  [11650, 11100, 11200, 11100, 11000],
      '예측값(N)':  [12100, 11800, 11200, 11100, 11150],}
cmdf = pd.DataFrame(cm, index=['실제값(p)', '실제값(n)'], columns=['예측값(Y)', '예측값(N)'])
cmdf

ValueError: ignored

ACCURACY

In [180]:
correct = merge_df[merge_df['yes/no'] == 'yes']
acc = len(correct)/len(merge_df)*100

print('Accuracy:', acc)

Accuracy: 61.904761904761905


CHI-SQUARE

In [ ]:
from scipy.stats
chi2, pvalue, dof, expected = scipy.stats.chi2_contingency(data)

chi = chi2_contingency()